In [4]:
with open('log1126_141.txt', 'r') as f:
    all_logs = f.readlines() 

In [5]:
import re

dataset_list = []
lr_list = []
epochs_list = []
hidden_list = []
dropout_list = []
weight_decay_list = []
alpha_list = []
beta_list = []
gamma_list = []
norm_layers_list = []
orders_list = []
highest_train_list = []
highest_valid_list = []
final_train_list = []
final_test_list = []
delta_list = []

for logs in all_logs:
    if 'Namespace' in logs:
        dataset_list.extend(re.findall(r"dataset='(.*?)', delta", logs))
        delta_list.extend(re.findall(r"delta=(.*?), directed", logs))
        lr_list.extend(re.findall(r"lr=(.*?), method", logs))
        epochs_list.extend(re.findall(r"epochs=(.*?), gamma", logs))
        dropout_list.extend(re.findall(r"dropout=(.*?), epochs", logs))
        weight_decay_list.extend(re.findall(r"weight_decay=(.*?)\)", logs))
        alpha_list.extend(re.findall(r"alpha=(.*?), beta", logs))
        beta_list.extend(re.findall(r"beta=(.*?), cached", logs))
        orders_list.extend(re.findall(r"orders=(.*?), orders_func_id", logs))
        gamma_list.extend(re.findall(r"gamma=(.*?), gat_heads", logs))
        norm_layers_list.extend(re.findall(r"norm_layers=(.*?), num_layers", logs))
        hidden_list.extend(re.findall(r"hidden_channels=(.*?), hops", logs))
        
    if 'Highest Train' in logs and ',' in logs:
        highest_train_list.extend(re.findall(r"Highest Train: (.*?),", logs))
    
    if 'Highest Valid' in logs and ',' in logs:
        highest_valid_list.extend(re.findall(r"Highest Valid: (.*?),", logs))
        
    if 'Final Train' in logs and ',' in logs:
        final_train_list.extend(re.findall(r"Final Train: (.*?),", logs))
        
    if 'Final Test' in logs and ',' in logs:
        final_test_list.extend(re.findall(r"Final Test: (.*?),", logs))


highest_train_list.append('0')
highest_valid_list.append('0')
final_train_list.append('0')
final_test_list.append('0')
        

In [6]:
import pandas as pd
import os

d = {
    'dataset': dataset_list,
    'lr': lr_list,
    'epoch': epochs_list,
    'hidden': hidden_list,
    'dropout': dropout_list,
    'weight_decay': weight_decay_list,
    'alpha': alpha_list,
    'beta': beta_list,
    'gamma': gamma_list,
    'delta': delta_list,
    'norm_layers': norm_layers_list,
    'orders': orders_list,
    'high_train': highest_train_list,
    'high_valid': highest_valid_list,
    'final_train': final_train_list,
    'final_test': final_test_list
}

for k, v in d.items():
    print(k, len(v))

csv_name = 'results/large_dataset_tuning14.csv'
df = pd.DataFrame.from_dict(d)
df = df.sort_values(['final_test']).reset_index(drop=True)
# if os.path.exists(csv_name):
#     df.to_csv(csv_name, mode='a', header=False)
# else:
#     df.to_csv(csv_name)
df[-20:]

dataset 6
lr 6
epoch 6
hidden 6
dropout 6
weight_decay 6
alpha 6
beta 6
gamma 6
delta 6
norm_layers 6
orders 6
high_train 6
high_valid 6
final_train 6
final_test 6


,dataset,lr,epoch,hidden,dropout,weight_decay,alpha,beta,gamma,delta,norm_layers,orders,high_train,high_valid,final_train,final_test
0,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.9,0.5,1,1,0,0,0,0
1,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.9,0.5,1,2,94.44,90.69,94.00,90.67
2,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.9,0.5,1,3,94.30,90.72,93.59,90.71
3,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.9,0.5,2,2,95.08,90.71,94.28,90.72
4,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.9,0.5,2,1,95.15,90.72,94.24,90.74
5,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.9,0.5,2,3,94.97,90.83,94.08,90.84
